In [1]:
import data
import model
import numpy as np
from model import build_sequential_model, create_clf

Using Theano backend.


In [2]:
data_folder = "."
X, y, _, _, _ = data.load(data_folder + "training.csv",
                          shuffle=True)
Xa, ya, wa, _, _ = data.load(data_folder + "check_agreement.csv",
                             shuffle=False, weight=True)
Xa_train, ya_train, _, _, _ = data.load(data_folder + "check_agreement.csv",
                                        shuffle=True)
Xc, _, _, mc, _ = data.load(data_folder + "check_correlation.csv",
                            shuffle=False, mass=True, test=True)
X, scaler = data.preprocess_data(X)
Xa, _ = data.preprocess_data(Xa, scaler)
Xa_train, _ = data.preprocess_data(Xa_train, scaler)
Xc, _ = data.preprocess_data(Xc, scaler)
X_test, _, _, _, ids = data.load("test.csv", test=True,
                                 ids=True)
X_test, _ = data.preprocess_data(X_test, scaler)

In [5]:
from keras.layers import PReLU, Dropout, Dense
def feature_extractor(input_size, output_size):
    return build_sequential_model([Dense(150, input_dim=input_size),
                                   PReLU(), Dropout(0.4), Dense(140),
                                   PReLU(), Dropout(0.35), Dense(130),
                                   PReLU(), Dropout(0.3), Dense(120),
                                   PReLU(), Dropout(0.27), Dense(output_size)],
                                  name="feature_extractor")


def label_classifier(input_size, name="label_classifier"):
    return build_sequential_model([Dense(100, input_dim=input_size),
                                  PReLU(), Dropout(0.3), Dense(80),
                                  PReLU(), Dropout(0.25), Dense(70),
                                  PReLU(), Dropout(0.2), Dense(2, activation='softmax')],
                                  name=name)

In [ ]:
n_models = 1
n_epochs = 130
probs = None
for i in range(n_models):
    from keras.utils import np_utils
    y_categorical = np_utils.to_categorical(y)
    np.random.seed(42)  # repeatability

    n_extracted_features = 100
    f = feature_extractor(X.shape[1], n_extracted_features)
    l = label_classifier(n_extracted_features)
    d = label_classifier(n_extracted_features, name="domain_classifier")

    transfering_ratio = 0.2  # 0.1
    # Learning on train

    m = create_clf(f, l, d, 0)
    metrics_callback = model.ShowMetrics(m, Xa, ya, wa, Xc, mc, X, y_categorical)
    f, l, d = model.fit_model(m, X, y_categorical, y_categorical, Xa, ya,
                              wa, Xc, mc, X, y,
                              epoch_count=int((1 - transfering_ratio) * n_epochs),
                              batch_size=128, validation_split=0.05, verbose=2,
                              callbacks=[metrics_callback])
    # Transfering to check_agreement
    ya_output = model.predict_model(m, np.array(Xa_train))
    steps = 3
    for step in range(steps):
        lam = np.linspace(0.1, 0.4, steps)[step] # np.random.choice(np.linspace(1, 10, 10))
        print('lambda = ', lam)
        m = create_clf(f, l, d, lam)
        f, l, d = model.fit_model(m, Xa_train, ya_output, np_utils.to_categorical(ya_train), Xa, ya, wa, Xc, mc, X, y,
                                  epoch_count=int(transfering_ratio * n_epochs / steps),
                                  batch_size=512, validation_split=0.5, verbose=2, show_metrics=True)

    # Output
    p = model.predict_probs_model(m, np.array(X_test))
    probs = p if probs is None else p + probs
probs /= n_models

Train on 64175 samples, validate on 3378 samples
Epoch 1/104
64175/64175 [==============================] - 7s - loss: 0.7989 - label_classifier_acc: 0.8299 - domain_classifier_acc: 0.8287 - val_loss: 0.6794 - val_label_classifier_acc: 0.8760 - val_domain_classifier_acc: 0.8757
Epoch 2/104
64175/64175 [==============================] - 6s - loss: 0.6516 - label_classifier_acc: 0.8717 - domain_classifier_acc: 0.8715 - val_loss: 0.6403 - val_label_classifier_acc: 0.8863 - val_domain_classifier_acc: 0.8866
Epoch 3/104
64175/64175 [==============================] - 18s - loss: 0.6213 - label_classifier_acc: 0.8776 - domain_classifier_acc: 0.8775 - val_loss: 0.5603 - val_label_classifier_acc: 0.8881 - val_domain_classifier_acc: 0.8887
Epoch 4/104
 3200/64175 [>.............................] - ETA: 18s - loss: 0.6099 - label_classifier_acc: 0.8875 - domain_classifier_acc: 0.8869

In [ ]:
data.save_submission(ids, probs, "grl_prediction.csv")
!zip -9 -r grl_prediciton.csv.zip grl_prediction.csv